In [ ]:
import pandas as pd
import numpy as np
import random
import string

# Fonction pour générer un mot aléatoire
def generate_random_word(length=10):
    letters = string.ascii_uppercase
    return ''.join(random.choice(letters) for i in range(length))

# Générer environ 4000 mots différents pour 'Underlying Name'
unique_words = [generate_random_word() for _ in range(4000)]

# Assurer que la taille du DataFrame est plus grande en répétant les mots
underlying_names = np.random.choice(unique_words, size=10000, replace=True)

# Générer des valeurs aléatoires pour 'Portfolio' et 'ProductTypeGroup'
portfolios = np.random.choice(['Portfolio1', 'Portfolio2', 'Portfolio3', 'Portfolio4'], size=10000, replace=True)
product_types = np.random.choice(['Type1', 'Type2', 'Type3', 'Type4'], size=10000, replace=True)

# Créer le DataFrame
df = pd.DataFrame({
    'Underlying Name': underlying_names,
    'Portfolio': portfolios,
    'ProductTypeGroup': product_types
})

# Afficher un échantillon du DataFrame
print(df.head())
print(f"Nombre de mots uniques dans 'Underlying Name' : {df['Underlying Name'].nunique()}")


: 